<font size="6"><b><i>Brian2</i> Simulation of Calcium Based Plasticity in a Spiking Model</font>

                                                                           Hanyu Li
                                                                           03/25/2016

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;

<IPython.core.display.Javascript object>

# Simulation

In [4]:
from brian_utils import *
from brian_simulator import *
%matplotlib inline
%cd ~/pypath/calcium-spike-model/

/home/hanyu/pypath/calcium-spike-model


In [5]:
params = {
    'cpre_0':0.1,
    'cpost_0':0.1,
    'rho_0':0.5,
    'c':0.2,
    'dummy':0.2,
    'Ipre':0,
    'Ipost':0,
    'w0':0.5,
    'w_EE':0.2,
    'w_IE':0.1,
    'w_II':-0.4,
    'w_EI':-0.4,
    #LIF specific constants,
    'tau_lif':26, #*ms
    'V_init':-60,
    'V_rest':-70,
    'V_reset':-70,
    'V_threshold':-50,
    'CM':0.001,
    'RM':20.0,
    'refrac':0,
    #Synapse model specific constants,
    'rho_init':0.019,
    'ca_initial':0,
    'ca_delay':4.61, #ms
    'Cpre':0.56175,
    'Cpost':1.23964,
    'eta':0,
    'tau_ca':22.6936,
    'theta_D':1,
    'theta_P':1.3,
    'gamma_D':331.909,
    'gamma_P':725.085,
    'taurho':346361, #*ms
    'taurho_fast':10, #*ms # dummy,
    'taupre':22,
    'taupost':22,
    'tau_ca':22, #*ms
    'rho_star':0.5,
    'D':4.6098,
    'baseline_I_ext_E':25,
    'baseline_I_ext_I':35,
    'mean_I_ext_E':40,
    'mean_I_ext_I':50,
    'sigma':20,
    'familiar_individual_sigma':9}

### Setup simulator

In [ ]:
# additively applied to params
param_diffs = {
    'cpre_0':0,
    'cpost_0':0,
    'rho_0':0,
    'c':0,
    'dummy':0,
    'Ipre':0,
    'Ipost':0,
    'w0':0,
    'w_EE':0,
    'w_IE':0,
    'w_II':0,
    'w_EI':0,
    #LIF specific constants,
    'tau_lif':0, #*ms
    'V_init':0,
    'V_rest':0,
    'V_reset':0,
    'V_threshold':0,
    'CM':0,
    'RM':0,
    'refrac':0,
    #Synapse model specific constants,
    'rho_init':0,
    'ca_initial':0,
    'ca_delay':0, #ms
    'Cpre':0,
    'Cpost':0,
    'eta':0,
    'tau_ca':0,
    'theta_D':0,
    'theta_P':0,
    'gamma_D':0,
    'gamma_P':0,
    'taurho':0, #*ms
    'taurho_fast':0, #*ms # dummy,
    'taupre':0,
    'taupost':0,
    'tau_ca':0, #*ms
    'rho_star':0,
    'D':0,
    'baseline_I_ext_E':0,
    'baseline_I_ext_I':0,

    #'mean_I_ext_E':0,
    #'mean_I_ext_I':0,
    #'sigma': 0,
    #'familiar_individual_sigma':0}

    'mean_I_ext_E':0,
    'mean_I_ext_I':0,
    'sigma':0,
    'familiar_individual_sigma':0}
    


# Control variables
simulation_length =8000
stair_length = 500
N_E = 1000
N_I = 250
sample = 10
input_flag = '_fam_fam_nov_'
resets = 1
debug = True

sim = Brian_Simulator(simulation_length=simulation_length, stair_length=stair_length,N_E=N_E,N_I=N_I,sample=sample,
         params=params, debug=debug)





# prepare result variables
spike_dict = build_real_value_spike_dict(params,param_diffs) # store spike trains for each parameter set
param_trial_num = len(spike_dict)
binned_rate_E = np.zeros((simulation_length * 10, param_trial_num))
binned_rate_I = np.zeros((simulation_length * 10, param_trial_num))
rho = np.zeros((N_E, simulation_length, param_trial_num))
mean_rate_shift =np.zeros((param_trial_num,1))


if param_trial_num == 1:
    mode = 'cpp_standalone'
else:
    mode = 'cython'


t = arange(simulation_length)

### Run simulator for each parameter set(only 1 in this demo)

In [ ]:

for ind, key in enumerate(spike_dict):
    cpp_directory = 'output_'+str(ind)
    (I_ext_E, I_ext_I, binned_rate_E[:,ind], binned_rate_I[:,ind], rho[:,:,ind], spike_dict[key]) = sim.run(key, mode=mode, input_flag=input_flag, resets=resets, cpp_directory=cpp_directory)


### Analysis

In [ ]:

(R_fam_E, R_fam_I, R_nov_E, R_nov_I) = unpack_raw_data('data/Data_Sheinberg_Neuron2012_FiringRates.mat')
analyse_all_parameter_sets(t, I_ext_E, spike_dict,R_fam_E,R_nov_E, params) 
visualize_all(I_ext_E, binned_rate_E, rho, t, resets, spike_dict.keys(), input_flag)
show()